# Bring your own containers to Braket Hybrid Jobs

Amazon Braket has pre-configured containers for executing Amazon Braket Hybrid Jobs, which are sufficient for many use cases involving the Braket SDK and PennyLane. However, if we want to use custom packages outside the scope of pre-configured containers, we will need to supply a custom-built container. In this tutorial, we show how to use Braket Hybrid Job to train a quantum machine learning model using BYOC (Bring Your Own Container).

<div class="alert alert-block alert-info">
<b>Note:</b> Building the docker image in this notebook may require 4GB of memory. If you plan to use an Amazon Braket notebook instance to run this notebook, it is suggested to use a larger notebook instance, for example ml.t3.large.
</div>

## 2 Prepare custom container
When we submit a quantum hybrid job, Amazon Braket starts a hybrid job instance based on EC2 and spins up a container to run our script. The environment is defined by the provided container instead of the local console where the hybrid job is submitted. If no container image is specified when submitting a hybrid job, the default container is the base Braket container. See the [developer guide](https://docs.aws.amazon.com/braket/index.html) for the configuration of the base container.

Amazon Braket Hybrid Jobs provides three pre-configured containers for different use cases. See the [developer guide](https://docs.aws.amazon.com/braket/index.html) for the configuration of pre-configured containers. In this example, the spaCy package is not supported in any of the three containers. One option is to install the package through <code>pip</code> at the beginning of the algorithm script.  

In [4]:
# from pip._internal import main as pipmain
# pipmain(["install", "spacy"])

When the problem size is small, we can manage to use <code>pip</code> to configure the environment. However, for large-scale applications, we expect that this method would quickly become infeasible. Braket Hybrid Job provides the "bring your own container (BYOC)" option to help you manage the environment of your hybrid job. BYOC not only allows us to define what Python packages are available, but to configure those settings that are hard to do by <code>pip</code> or Python alone. In the following, we go through the steps of building our own container.

### Preparation 1: Docker 
To build and upload our custom container, we must have [Docker](https://docs.docker.com/get-docker/) installed. Amazon Braket Notebook Instance has Docker pre-installed. This step can be skipped if you are using the terminal of a Braket Notebook Instance.

### Preparation 2: Dockerfile
A Dockerfile defines the environment and the software in the containers. We can start with the base Braket container and add packages according to our needs. For our quantum text classifier, we use the Dockerfile below. The first line in the dockerfile specifies the container template. We build our container upon the base Braket container. The rest of the file is to install the required packages (PennyLane and SpaCy etc.).

In [1]:
!cat dockerfile

FROM 011528273261.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-base-jobs:latest
RUN python3 -m pip install --upgrade pip

RUN python3 -m pip install qiskit \
                           pyscf \
                           qiskit-nature \
                           qiskit-braket-provider \
                           qiskit-algorithms


### Preparation 3: Initial script
An initial script is the script that will be executed when a container starts. For this example notebook, we build our container based on the base Braket container which already includes an initial script to run Braket Hybrid Jobs. Therefore, we do not need to provide an initial script. The initial script configures the paths for container image and for user code. It sets up a container and downloads the algorithm script to run in the container. It also handles errors and logs error messages. See the developer guide for more information about the initial script associated with the base Braket container.

### Preparation 4: Create ECR
Amazon Elastic Container Registry (ECR) is a fully managed Docker container registry. Follow the [instructions](https://docs.aws.amazon.com/AmazonECR/latest/userguide/repository-create.html) to create a "private" repository using ECR console. For this example, we name our repository "amazon-braket-my-qtc" (my quantum text classifier).

Alternatively, following these [instructions](https://docs.aws.amazon.com/cli/latest/reference/ecr/create-repository.html), we can also create an ECR repository using the AWS CLI.

<b>Important</b>: The Amazon Braket managed policies only grant read access to repositories with the prefix `amazon-braket`. <b>In order to create (and later, push to) a repository, or to access repositories which are not prefixed with `amazon-braket`, you will need to attach additional permissions to your IAM identity.</b> If you are running this notebook on an Amazon Braket notebook instance, you may attach the [AmazonEC2ContainerRegistryFullAccess](https://docs.aws.amazon.com/AmazonECR/latest/userguide/security-iam-awsmanpol.html#security-iam-awsmanpol-AmazonEC2ContainerRegistryFullAccess) policy to the IAM role you specified when creating the notebook instance.

<b>Important: By default, Braket Hybrid Jobs can only access repositories with names beginning in `amazon-braket`.</b> If you would like to access a repository with a different prefix, you will need to pass an IAM role with access to that repository using the `role_arn` argument of the `AmazonQuantumJob.create()` function.

In [2]:
!aws ecr create-repository --repository-name amazon-braket-inhibitq-qiskit

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:eu-west-2:011528273261:repository/amazon-braket-inhibitq-qiskit",
        "registryId": "011528273261",
        "repositoryName": "amazon-braket-inhibitq-qiskit",
        "repositoryUri": "011528273261.dkr.ecr.eu-west-2.amazonaws.com/amazon-braket-inhibitq-qiskit",
        "createdAt": 1734085929.434,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


Now that we have finished the prerequisites, it's time to build our container!

### Action 1: Log into AWS CLI and Docker
If you haven't already, follow the [instructions](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html) to configure your AWS credentials using the AWS CLI. Then, run the following snippet in the terminal to log into Docker. Replace all &lt;XXX&gt; below with your own credentials. You will see "Login Succeeded" twice when it's done.

<code>aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 292282985366.dkr.ecr.us-west-2.amazonaws.com
aws ecr get-login-password --region &lt;YOUR_AWS_REGION&gt; | docker login --username AWS --password-stdin &lt;YOUR_ACCOUNT_ID&gt;.dkr.ecr.&lt;YOUR_AWS_REGION&gt;.amazonaws.com</code>

Note that the commands log into Docker twice. The first command logs into the public registry containing the Braket base container that you will base your Docker image on. The second command logs into your private registry. 

If the terminal does not support interactive login, you can also run the following commands to log in.

<code>docker login -u AWS -p $(aws ecr get-login-password --region us-west-2) 292282985366.dkr.ecr.us-west-2.amazonaws.com
docker login -u AWS -p $(aws ecr get-login-password --region &lt;YOUR_AWS_REGION&gt;) &lt;YOUR_ACCOUNT_ID&gt;.dkr.ecr.&lt;YOUR_AWS_REGION&gt;.amazonaws.com</code>

aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 292282985366.dkr.ecr.us-west-2.amazonaws.com
aws ecr get-login-password --region eu-west-2 | docker login --username AWS --password-stdin 011528273261.dkr.ecr.eu-west-2.amazonaws.com

### Action 2: Build and push the image
Go to the folder containing your Dockerfile. Run the lines below to build and push the image to your ECR. Remember to replace all &lt;XXX&gt; in the code with your own credentials. When it completes, you will see all layers are pushed in the terminal, and a new image will appear in our ECR console under the "amazon-braket-my-qtc" repository. If running into memory error when building an image due to the size of the language model, you can increase the memory limit in Docker.<br>
<code>docker build -t dockerfile .
docker tag dockerfile:latest &lt;YOUR_ACCOUNT_ID&gt;.dkr.ecr.&lt;YOUR_AWS_REGION&gt;.amazonaws.com/amazon-braket-my-qtc:latest
docker push &lt;YOUR_ACCOUNT_ID&gt;.dkr.ecr.&lt;YOUR_AWS_REGION&gt;.amazonaws.com/amazon-braket-my-qtc:latest
</code>
Once the container image is uploaded, it is ready to be used in a Braket Hybrid Job!

docker build -t dockerfile .
docker tag dockerfile:latest 011528273261.dkr.ecr.eu-west-2.amazonaws.com/amazon-braket-inhibitq-qiskit:latest
docker push 011528273261.dkr.ecr.eu-west-2.amazonaws.com/amazon-braket-inhibitq-qiskit:latest

### The build-and-push shell script
The above procedure walks you through the steps to create a container image. This procedure can be automated in a shell script. The [example script](build_and_push.sh) is provided in the same folder of this notebook. The script automatically formulates the commands to build and to push the container image to the ECR repository you assign. If the repository does not exist, it creates one. To use this shell script, the IAM identity that runs the script requires permissions to create the repository and push the image to the repository. If your IAM identity doesn't have these permissions, you can attach the [AmazonEC2ContainerRegistryFullAccess](https://docs.aws.amazon.com/AmazonECR/latest/userguide/security-iam-awsmanpol.html#security-iam-awsmanpol-AmazonEC2ContainerRegistryFullAccess) policy or select only the relevant permissions if you require granular control.

Assuming "amazon-braket-my-qtc" is your repository name, all you need to do is to prepare the Dockerfile and run the command<br>
<code>sh build_and_push.sh amazon-braket-my-qtc
</code>
in the terminal. Alternatively you can uncomment and run the following cell:

In [7]:
# !sh build_and_push.sh amazon-braket-my-qtc

## 3 Submit your hybrid job
Now that we have prepared an algorithm script and the container for the hybrid job, we can submit the hybrid job to AWS using <code>AwsQuantumJob.create</code>. Remember to provide the container we just created via the <code>image_uri</code> keyword.

In [6]:
from braket.aws import AwsQuantumJob
from braket.devices import Devices

image_uri = "011528273261.dkr.ecr.eu-west-2.amazonaws.com/amazon-braket-inhibitq-qiskit:latest"

job = AwsQuantumJob.create(
    device=Devices.Amazon.SV1,
    source_module="algorithm_script.py",
    entry_point="algorithm_script:main",
    wait_until_complete=False,
    job_name="my-aws-job2",
    image_uri=image_uri,
)

In [9]:
print("Quantum Task Summary")
print(job.result()["VQE"])
print(
    "Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2)."
)
#print(
#    f"Estimated cost to run quantum tasks in this hybrid job: {job.result()['estimated cost']} USD"
#)

Quantum Task Summary
{'eigenvalue': -1.2038862681539855, 'optimal_parameters': [1.6109092882171812, -4.304987725882563, -1.3270790527920857, -2.921281739203198, 0.4877379876708545, 5.7068604959655485, -5.713053151059345, -3.4438186206813], 'optimal_point': [1.6109092882171812, -4.304987725882563, -1.3270790527920857, -2.921281739203198, 0.4877379876708545, 5.7068604959655485, -5.713053151059345, -3.4438186206813], 'optimal_value': -1.2038862681539855}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
